# Title: **PyTorch** 라이브러리를 이용한 **ResNet** 구현 

### Step 0) PyTorch 패키지 import

In [ ]:
import torch

## 데이터 전처리 관련 패키지
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## 딥러닝 모델 생성 관련 패키지
import torch.nn as nn

## 딥러닝 모델 업데이트 알고리즘 관련 패키지
import torch.optim as optim

## 다차원 데이터 처리를 위한 라이브러리
import numpy as np

## 데이터 시각화를 위한 라이브러리
import matplotlib.pyplot as plt

0) 데이터 전처리 관련 패키지

 - `torchvision.datasets`: 다양한 dataset들을 포함하고 있는 패키지 (https://pytorch.org/vision/stable/datasets.html)

 - `torchvision.transforms`: 데이터 변형을 위한 라이브러리

 - `torch.utils.data.DataLoader`: 데이터를 `batch size` 크기로 분할해주는 라이브러리


1) 딥러닝 모델 생성 관련 패키지

  - `torch.nn (nn)`: 딥러닝 모델을 생성할 때 층(layer) 관련 함수들(예: Linear, Conv2d, MaxPool2d, ReLU, BatchNorm2d)을 모아놓은 패키지

  - `torch.nn.Module (Module)`: 생성한 층들을 연합하여 하나의 모델로 구성할 때 사용하는 클래스

2) 딥러닝 모델 학습 관련 패키지

  - `torch.optim (optim)`: 모델 업데이트 알고리즘(`optimizer`)들을 모아놓은 패키지 

 ### Step 1) 데이터 불러오기 및 전처리 

 - **Data Augmentation (데이터 증강)**
  
  - 원본 데이터를 변형하여 데이터의 개수를 늘리는 기법 (**데이터 다양화**)
      - 변형의 예) 회전(rotation), 반전 or 대칭(flip), 잘라내기(crop)

      <img src="https://miro.medium.com/max/1400/0*LR1ZQucYW96prDte" width=700>

      - 참고) `ResNet` 학습에도 `Data Augmentation` 기법이 사용되었습니다! (https://arxiv.org/pdf/1512.03385.pdf)

  - `Data Augmenetation`을 사용하면 성능을 향상시킬 수 있을까?

    <img src='https://datadiving.dothome.co.kr/Deep%204_data%20augmentation.jpg' border='0'></a>

    **Flip**

    - 왼쪽만 바라보는 고양이가 train 때 주어졌다면 오른쪽을 보는 고양이는 test 때 맞추지 못할 수 있습니다.
    
    - 이 경우, 왼쪽만 바라보는 고양이를 좌우 대칭시켜 모두 학습에 사용하면 test 때 고양이가 어느 쪽을 보더라도 맞출 수 있게 됩니다.

    <img src='https://datadiving.dothome.co.kr/Deep%204_data%20augmentation%202.jpg' border='0'></a>

    **Crop**

    - 딥러닝 모델이 꼬리와 귀, 수염, 눈이라는 특징으로 고양이인지를 판단한다면 고양이가 침대 밑에 숨어있어 꼬리만 보이는 경우 고양이라고 판단하지 못할 수 있습니다.

    - 이 경우, 고양이의 각 부분을 잘 알려주기 위해 꼬리 부분만 잘라서 넣어준다면 성능이 더 좋아질 수 있을 것입니다.

    <img src='https://datadiving.dothome.co.kr/Deep%204_%EB%B0%9D%EA%B8%B0%20%EC%A1%B0%EC%A0%88.jpg' border='0'></a>

    **💡 밝기 조절**  
    - 만약 딥러닝 모델로 앱을 만들게 되면, 사진을 찍는 사람마다 빛의 양이 다르겠죠? 딥러닝이 동일하게 인식할 수 있도록 어두운 것부터 밝은 것까지 모든 사진의 밝기를 조절해서 넣어줍시다!    

 - PyTorch에서는 `torchvision.transforms` 라이브러리를 통하여 `data augmentation`을 쉽게 구현할 수 있습니다!

 - CIFAR10 데이터셋 불러오기

  - `ResNet`에서는 

    - 이미지 픽셀의 평균 값을 0으로 만든 후 (`per-pixel mean subtracted`)

    - 4개의 픽셀들을 모서리 부분마다 추가해주고 (`4 pixels are padded on each side`)

    - 랜덤하게 좌우 대칭시킨 상태에서 (`horizontal flip`)
    
    - 32 * 32 크기의 이미지를 랜덤하게 잘라내서 학습데이터로 사용하였습니다 (`32 * 32 crop is randomly sampled`).

  - PyTorch의 이미지 변환 관련 패키지인 `torchvision.transforms`를 이용하여 위 `data augmentation`을 구현해봅시다.

    - `RandomCrop(size, padding)`

    - `RandomHorizontalFlip()`

In [ ]:
from torchvision import datasets, transforms

train_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        transforms.RandomCrop(size=32, padding=4),
        transforms.RandomHorizontalFlip()
    ]
)
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


 - 뿐만 아니라, `DataLoader` 함수를 통하여 전체 데이터셋을 `배치 (batch)` 단위로 쪼갤 수 있습니다.

  - ResNet에서는 `batch size=128`로 설정하였습니다.

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

### Step 2) **ResNet** 생성

 - PyTorch에서는 `torch.nn` 라이브러리 내의 함수들과 `클래스 (class)` 용법을 사용하여 인공신경망을 생성합니다.

    - **nn.Conv2d**: 합성곱 계층

    - **nn.MaxPool2d**: 풀링 계층

    - **nn.Flatten**: 다차원 데이터를 1차원으로 평탄화 시켜주는 계층

    - **nn.Linear**: 전결합계층

    - **nn.BatchNorm2d**: 배치 정규화

 - **Batch Normalization (배치 정규화)**
  
  - `Vanishing Gradient Problem`: `Backpropagation`에 의해 gradient가 앞단으로 전파되면서 점점 옅어지게 되어 너무 작아져서 **소멸**하게 되는 문제

    <img src="https://t1.daumcdn.net/cfile/tistory/997E1B4C5BB6EAF239" width=700>

    - Layer를 통과할 때마다 `활성함수의 미분값(gradient)`이 계속 곱해지는데, 기울기가 완만한 영역의 입력값이 주어지는 경우 앞단의 `gradient`가 아주 작아 모델이 업데이트 되지 않는 문제

  - `Internal Covariate Shift`: Layer를 통과할 때마다 입력값의 분포가 변화하는 현상

    <img src='https://datadiving.dothome.co.kr/Deep%204_internal%20Covariate%20Shift%202.jpg' width=700>

    - Layer가 깊어질수록 이전에 더 많은 layer를 통과하기 때문에 입력값의 분포가 많이 변하게 된다.

    <img src="https://gaussian37.github.io/assets/img/dl/concept/batchnorm/3.png" width=700>

    - 입력값의 분포가 점점 변형되어 `gradient`가 작은 영역에 도달하게 된다면 `vanishing gradient`문제가 발생한다.

  - `배치 정규화 (Batch Normalization)`: 각 레이어를 통과할 때마다 **정규화 (normalization)**하는 레이어를 두어 입력값의 분포가 변형되지 않도록 조절하는 알고리즘

    <img src="https://gaussian37.github.io/assets/img/dl/concept/batchnorm/4.png" width=700>

    - 배치 단위로 실시하기 때문에 `batch normalization`이라고 부릅니다.

    - 활성함수의 입력값의 분포를 조절해주는 것이 목적이므로 **활성함수** 앞에 넣어준다.

    - 알고리즘 테이블

    <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcFYkLE%2FbtqEcUnlXKy%2FZbGZNjObjo2gL2xss8zYzk%2Fimg.png" width=500>

    - $\gamma, \beta$로 `scale and shift`를 해주는 이유?

    <img src="https://miro.medium.com/max/1400/1*0yhJ7DbhOX-tRUseljjYoA.png" width=700>

 - **ResNet**
  - ResNet에서의 key idea는 2개의 layer를 통과할 때마다 `skip connection`을 넣어주어`backpropagation` 과정에서 **gradient가 잘 흐르도록 하는 것**입니다.
 
    <img src="https://images.velog.io/images/junyoung9696/post/3137e50c-b52f-4cdd-8ae8-2faf497efe84/r10.png" width=500>

 #### Step 2-1) Convolution Block 구성
  
 - `ResNet`에서는 **$3 \times 3$ convolution + batch normalization + relu** 계층들이 반복해서 쌓인 형태이고, 위 계층을 2번 통과할 때마다 **skip (shortcut) connection**을 추가해줍니다.

 -  우선, 2개의 $3 \times 3$ convolution + batch normalization + relu로 구성된 block을 생성해주는 함수를 코딩해봅시다.

  - `__init__(self)` 함수에는 하나의 block을 구성하는 계층들을 적어주시고,

  - `forward(self, x)` 함수에는 위에서 정의한 계층들을 엮어서 입력 x가 주어졌을 때의 출력값을 적어주시면 됩니다.

  - `shortcut connection` 구현 시 원래의 입력 값과 convolution을 통과한 출력 사이의 shape이 다른 경우, **$1 \times 1$ convolution**을 추가해줍시다.

In [ ]:
from torch.nn.modules.batchnorm import BatchNorm2d
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride):
    super().__init__()
    
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=stride)
    self.bn1 = nn.BatchNorm2d(num_features=out_channels)
    self.relu1 = nn.ReLU()

    self.conv2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=stride)
    self.bn2 = nn.BatchNorm2d(num_features=out_channels)
    self.relu2 = nn.ReLU()

    if (in_channels == out_channels) and (stride == 1):
      self.shortcut = nn.Sequential()
    else:
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1,stride=stride),
            nn.BatchNorm2d(num_features=out_channels)
        )
  def forward(self, x):
    h = self.relu1(self.bn1(self.conv1(x)))
    h = self.bn2(self.conv2(x))
    h = self.relu2(h +self.shortcut(x))
    
    return h

#### Step 2-2) ResNet 생성

 - 위에서 구성한 `ConvBlock`을 쌓아 `ResNet`을 생성해봅시다. 
  
  - `__init__(self)` 함수에는 ResNet을 구성하는 계층들을 적어주시고,

  - `forward(self, x)` 함수에는 위에서 정의한 계층들을 엮어서 입력 x가 주어졌을 때의 출력값을 적어주시면 됩니다. 

  <img src="https://blog.kakaocdn.net/dn/bQfaUX/btqYAtD1KcX/Zdc4DLFzR9SoJYBlO6M1uK/img.png" width=900>

In [ ]:
from torch.nn.modules.conv import Conv2d
import torch.nn.functional as F

class ResNet(nn.Module):
  def __init__(self):
    super().__init__()

    ## 특징 추출 [3, 32, 32] -> [512, 4, 4]
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(num_features=64)
    )

    self.block1 = nn.Sequential(
        ConvBlock(in_channels=64, out_channels=64, stride=1),
        ConvBlock(in_channels=64, out_channels=64, stride=1)
    )

    self.block2 = nn.Sequential(
        ConvBlock(in_channels=64, out_channels=128, stride=2),
        ConvBlock(in_channels=128, out_channels=128, stride=1)
    )

    self.block3 = nn.Sequential(
        ConvBlock(in_channels=128, out_channels=256, stride=2),
        ConvBlock(in_channels=256, out_channels=256, stride=1)
    )

    self.block4 = nn.Sequential(
        ConvBlock(in_channels=256, out_channels=512, stride=2),
        ConvBlock(in_channels=512, out_channels=512, stride=1)
    )

    self.pool = nn.AvgPool2d(kernel_size=4)  # [512, 4, 4] -> [512, 1, 1]

    ## 분류
    self.flatten = nn.Flatten() # [512, 1, 1] -> [512]
    self.fc = nn.Linear(in_features=512, out_features=10) # 파이토치에서는 교차엔트로피 구하는 함수 내에 1) one-hot encoding, 2) softmax가 포함되어 있습니다.

  def forward(self, x):
    h = F.relu(self.conv1(x))

    h = self.block1(h)
    h = self.block2(h)
    h = self.block3(h)
    h = self.block4(h)

    h = self.pool(h)
    h = self.flatten(h)
    logits = self.fc(h)

    return logits

 - `torchsummary.summary` 함수를 이용하여 의도에 맞게 ResNet이 생성되었는지 확인해봅시다!

In [ ]:
from torchsummary import summary

resnet = ResNet()
resnet.to("cuda")
summary(resnet, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,928
       BatchNorm2d-4           [-1, 64, 32, 32]             128
              ReLU-5           [-1, 64, 32, 32]               0
            Conv2d-6           [-1, 64, 32, 32]          36,928
       BatchNorm2d-7           [-1, 64, 32, 32]             128
              ReLU-8           [-1, 64, 32, 32]               0
         ConvBlock-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,928
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]          36,928
      BatchNorm2d-14           [-1, 64,

### Step 3) 학습 (Training)

 - 손실 함수 (Loss) 및 업데이트 알고리즘 (Optimizer) 설정
    
    - 손실함수로는 **교차엔트로피오차 (cross entropy)**를
    
    - 업데이트 알고리즘으로는 **SGD (Stochastic Gradient Descent)** optimizer를 사용하겠습니다.

        - `weight_decay=0.0005, momentum=0.9`로 설정하겠습니다. 

        - 학습 초기에는 `learning_rate=0.1`로 설정하고, **특정 epoch에 도달하면 learning rate를 10배로 줄여주는** `scheduler`를 사용하겠습니다. 

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=resnet.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,120], gamma=0.1)

 👨🏻‍🏫 학습 순서

  1. 입력 (x)에 대한 결과 예측 -> y_pred

  2. y_pred와 정답 (y_true)을 비교하여 손실함수의 값 계산 -> **criterion(y_pred, y_true)**

  3. 손실함수의 값이 작아지는 방향으로 네트워크 업데이트 -> **optimizer.step()**

In [ ]:
from os import X_OK
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import tqdm

def evaluate(model, data_loader):
  correct = 0
  total = 0

  with torch.no_grad():
    for batch in data_loader:
      images, labels = batch
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)

      total += labels.size(0)
      correct += (predicted == labels).sum().item()
    
    acc = 100 * correct / total
  
  return acc

def train(model, criterion, optimizer, scheduler, train_loader, epochs, test_loader):
  best_acc = 0
  
  for epoch in range(epochs):  
    train_loss = 0
    
    correct = 0
    total = 0

    for idx, batch in enumerate(tqdm.tqdm(train_loader)):
      x_batch, y_batch = batch
      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)
      
      optimizer.zero_grad()
        
      # 입력에 대한 결과 예측
      logits = model(x_batch)

      # 예측값과 정답을 비교하여 손실함수의 값 계산
      loss = criterion(logits, y_batch)

      # 손실함수의 값이 작아지는 방향으로 네트워크 업데이트
      loss.backward()
      optimizer.step()

      train_loss += loss.item()
      

      # 평가 - Accuracy 측정
      _, y_pred = torch.max(logits, dim=1)
      correct += torch.sum(torch.eq(y_pred, y_batch)).item()
      total += y_pred.size(0)

      if idx % 150 == 0:
        print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(idx+1), 100.*correct/total, correct, total))

    # Learning rate scheduling
    scheduler.step()

    # test_data에 대한 성능 평가
    test_acc = evaluate(model, test_loader)
    print(f'Epochs:{epoch + 1}, Test Accuracy:{test_acc}')

    ## test_data에 대하여 가장 좋은 성능을 보이는 모델을 저장


In [ ]:
train(model=resnet, criterion=criterion, optimizer=optimizer, scheduler=scheduler, train_loader=train_loader, epochs=200, test_loader=test_loader)

  1%|          | 2/391 [00:01<03:14,  2.00it/s]

Loss: 2.477 | Acc: 7.812% (10/128)


 39%|███▉      | 153/391 [00:13<00:20, 11.71it/s]

Loss: 2.560 | Acc: 21.290% (4115/19328)


 77%|███████▋  | 303/391 [00:26<00:08, 10.92it/s]

...

100%|██████████| 391/391 [00:34<00:00, 11.49it/s]


Epochs:170, Test Accuracy:93.46


  1%|          | 3/391 [00:00<00:51,  7.49it/s]

Loss: 0.005 | Acc: 100.000% (128/128)


 22%|██▏       | 85/391 [00:07<00:26, 11.70it/s]

In [ ]:
train(model=resnet, criterion=criterion, optimizer=optimizer, scheduler=scheduler, train_loader=train_loader, epochs=200, test_loader=test_loader)

  1%|▋                                                                                 | 3/391 [00:03<05:20,  1.21it/s]

Loss: 2.392 | Acc: 11.719% (15/128)


 39%|███████████████████████████████▎                                                | 153/391 [00:12<00:15, 15.59it/s]

Loss: 2.580 | Acc: 16.241% (3139/19328)


 77%|█████████████████████████████████████████████████████████████▉                  | 303/391 [00:22<00:05, 15.35it/s]

...

Loss: 0.002 | Acc: 100.000% (19328/19328)


 77%|█████████████████████████████████████████████████████████████▉                  | 303/391 [00:23<00:06, 14.63it/s]

Loss: 0.002 | Acc: 99.997% (38527/38528)


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:29<00:00, 13.14it/s]


Epochs:200, Test Accuracy:94.27


In [ ]:
net = torch.load("./trained_model.pt")
evaluate(net, test_loader)

94.48